In [19]:
from sklearn.cluster import KMeans
import pymongo
from pymongo import MongoClient
import numpy as np
from matplotlib import pyplot as plt
from firebase import firebase
import json
import pickle
import math


In [23]:
client = MongoClient()
db=client.nh4
lngs=[]
lats=[]
ids=[]
rtts=[]
for issue in db.issues.find():
    lats.append(issue["lat"])
    lngs.append(issue["lng"])
    ids.append(issue["id"])
    rtts.append(issue["request_type_title"])


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

In [22]:
clusters = {}
for rtt in list(set(rtts)):
     if not rtt=="Post to Neighbors":
        print rtt
        clusters[rtt]={}
        rttind=(np.array(rtts)==rtt).nonzero()[0]
        rttlngs = np.array(lngs)[rttind]
        rttlats = np.array(lats)[rttind]
        cluster = KMeans(n_clusters=len(rttind)/10)
        cluster.fit(np.array([rttlngs,rttlats]).transpose())
        cluster_centers = cluster.cluster_centers_
        cluster_labels = cluster.labels_
        for cluster in list(set(cluster_labels)):
            clusters[rtt][str(cluster)]=[]
            clusterind=rttind[np.array(cluster_labels)==cluster]
            for ID in np.array(ids)[np.array(clusterind)]:
                issue = db.issues.find_one({"id":ID})
                clusters[rtt][str(cluster)].append(issue)
    #plt.figure()
    #for i in range(len(cluster_centers)):
    #    q = cluster_centers[i]
        #plt.plot(q[0],q[1],'rx')
    #print len((cluster_labels==i).nonzero()[0])
        #for j in (np.array(rtts)==rtt).nonzero()[0]:
    #    for k in np.array(cluster_labels==i).nonzero()[0]:
        #print j
    #        plt.plot(rttlngs[k],rttlats[k],'bo')
        #print j
    #    plt.plot(q[0],q[1],'rx')
    #    plt.show()
#plt.show()
# for i in range(len(cluster_labels)):
#        db.issues.update({"id",ids[i]},{"$set":{"cluster":cluster_labels[i]}})

NameError: name 'rtts' is not defined

In [109]:
def sort_clusters(clustersd,scores):
    sc = [i[0] for i in sorted(zip(clustersd, scores), key=lambda l: l[1], reverse=True)]
    return sc

In [116]:
for rtt,clustersd in clusters.items():
    clusters[rtt]=clusters.pop(rtt).values()
    scores=[]
    for i in range(len(clusters[rtt])):
        closed=0
        opened=0
        for j in range(len(clusters[rtt][i])):
            if clusters[rtt][i][j]["status"]=="Open":
                opened+=1
            else:
                closed+=1
            meta=clusters[rtt][i][j]
            geo=[clusters[rtt][i][j]["lng"],clusters[rtt][i][j]["lat"]]
            clusters[rtt][i][j]={"meta":meta,"geo":geo}
        scores.append(math.sqrt(float(opened+closed))*float(opened)/float(closed+opened))
    clusters[rtt]=sort_clusters(clusters[rtt],scores)

In [113]:
clusters["Street Sweeping"][0][0]

{'geo': [-72.9190461947953, 41.3028745496296],
 'meta': {u'acknowledged_at': u'2015-05-26T15:59:41-04:00',
  u'address': u'210 Wooster Street New Haven, Connecticut',
  u'closed_at': u'2015-05-28T14:05:53-04:00',
  u'comment_url': u'https://seeclickfix.com/api/v2/issues/1670857/comments',
  u'created_at': u'2015-05-26T15:08:54-04:00',
  u'description': u'The no parking signs posted on trees for monthly street sweeping are still up along Wooster St from a couple of weeks ago. Please take down or they will confuse people this Friday.',
  u'flag_url': u'https://seeclickfix.com/api/v2/issues/1670857/flag',
  u'html_url': u'https://seeclickfix.com/issues/1670857',
  u'id': 1670857,
  u'lat': 41.3028745496296,
  u'lng': -72.9190461947953,
  u'media': {u'image_full': None,
   u'image_square_100x100': None,
   u'representative_image_url': u'https://crm.seeclickfix.com/assets/categories_trans/no-image.png',
   u'video_url': None},
  u'point': {u'coordinates': [-72.9190461947953, 41.302874549629

In [93]:
#with open("nhclusters.json","w") as f:
#    json.dump(clusters,f)
with open("ttclusters.json","w") as f:
    json.dump(clusters["Tree Trimming"],f)
    
#with open("nhclusters.pkl","w") as f:
 #   pickle.dump(clusters,f)

KeyboardInterrupt: 

In [115]:
with open("nhclusters.json","r") as f:
    clusters=json.load(f)

In [5]:
for j in clusters.values():
    for k in j.keys():
        print k

1
0
3
2
5
4
6
56
42
22
43
60
49
52
53
24
25
26
27
20
21
48
23
46
47
44
45
28
29
40
41
1
0
3
2
5
4
7
6
9
8
18
39
38
59
14
11
10
13
12
15
58
17
16
19
32
31
30
51
36
35
34
33
55
37
54
57
50
132
131
130
24
25
26
27
20
21
22
23
28
29
0
4
8
87
120
121
122
123
124
125
126
127
128
129
59
58
55
54
57
56
51
50
53
52
115
114
88
89
111
110
113
112
82
83
80
81
119
118
84
85
3
7
108
109
102
103
100
101
106
107
104
105
39
38
33
32
31
30
37
36
35
34
60
61
62
63
64
65
66
67
68
69
2
6
99
98
91
90
93
92
95
94
97
96
11
10
13
12
15
14
17
16
19
18
117
116
48
49
46
86
44
45
42
43
40
41
1
5
9
77
76
75
74
73
72
71
70
79
78
47
165
133
132
131
130
137
136
135
134
139
138
166
24
25
26
27
20
21
22
23
160
28
29
161
0
4
8
163
120
121
122
123
124
125
126
127
128
129
167
59
58
55
54
57
56
51
50
53
52
164
201
199
198
200
195
194
197
178
191
190
193
192
115
114
88
89
111
110
113
112
82
83
80
81
119
118
84
85
141
3
7
108
109
102
103
100
101
106
107
104
105
39
38
33
32
31
30
37
36
35
34
162
176
60
61
62
63
64
65
66
67
68


In [117]:
with open("swclusters.json","w") as f:
    json.dump([clusters["Tree Trimming"],clusters["Potholes"],clusters["Sidewalks and Curb damage"],clusters["Traffic Signal   Pedestrian Signal"]],f)